In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:

Run LASSO with different L1 penalties.
Choose best L1 penalty using a validation set.
Choose best L1 penalty using a validation set, with additional constraint on the size of subset.
In the second notebook, you will implement your own LASSO solver, using coordinate descent.



In [3]:
import numpy as np
import pandas as pd



In [4]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int,
    'sqft_living15':float,'grade':int, 'yr_renovated':int, 'price':float, 
    'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 
    'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 
    'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}


In [5]:
sales=pd.read_csv('kc_house_data.csv',dtype=dtype_dict)


In [6]:
sales.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0


# Create new features

In [7]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

In [8]:
sales.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_living_sqrt,sqft_lot_sqrt,bedrooms_square,floors_square
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,...,0,98178,47.5112,-122.257,1340.0,5650.0,34.351128,75.166482,9.0,1.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,...,1991,98125,47.7210,-122.319,1690.0,7639.0,50.695167,85.099941,9.0,4.0


Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.

On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [9]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (l1_penalty) to the linear regression. (Other tools may have separate implementations of LASSO.) Note that it's important to set l2_penalty=0 to ensure we don't introduce an additional L2 penalty.



In [10]:
from sklearn import linear_model

In [11]:
model_all = linear_model.Lasso(alpha=5e2,normalize=True)
model_all.fit(sales[all_features],sales['price'])

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [12]:
print model_all.coef_
model_all.intercept_

[     0.              0.              0.            134.43931396      0.
      0.              0.              0.              0.              0.
  24750.00458561      0.          61749.10309071      0.              0.
     -0.              0.        ]


-218136.21403515909

In [13]:
features_coef_greater_than_zero=[]
for i in range(len(all_features)):
    if(model_all.coef_[i]>0):
        features_coef_greater_than_zero.append(all_features[i])

print features_coef_greater_than_zero
print ''


['sqft_living', 'view', 'grade']



In [14]:
#3. Quiz Question: Which features have been chosen by LASSO, i.e. which features were assigned nonzero weights?
print features_coef_greater_than_zero       


['sqft_living', 'view', 'grade']


4. To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets. Download the provided csv files containing training, validation and test sets.


In [15]:
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)


In [16]:
#Make sure to create the 4 features as we did in #1:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']



In [17]:
training.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_living_sqrt,sqft_lot_sqrt,bedrooms_square,floors_square
0,2487200875,20141209T000000,604000.0,4.0,3.0,1960.0,5000,1.0,0,0,...,0,98136,47.5208,-122.393,1360.0,5000.0,44.271887,70.710678,16.0,1.0
1,7237550310,20140512T000000,1225000.0,4.0,4.5,5420.0,101930,1.0,0,0,...,0,98053,47.6561,-122.005,4760.0,101930.0,73.620649,319.264780,16.0,1.0


5. Now for each l1_penalty in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, 
type np.logspace(1, 7, num=13).)
	•	Learn a model on TRAINING data using the specified l1_penalty. Make sure to specify normalize=True in the constructor:


In [18]:
#Learn a model on TRAINING data using the specified l1_penalty. Make sure to specify normalize=True in the constructor:
#Compute the RSS on VALIDATION for the current model (print or save the RSS)

l1_penalty_value = np.logspace(1, 7, num=13)
errors ={}
for l1_penalty in l1_penalty_value:
    model = linear_model.Lasso(alpha=l1_penalty,normalize=True)
    # fit model on training data
    model.fit(training[all_features],training['price'])
    predictions =model.predict(validation[all_features])
    residuals = predictions-validation['price']
    rss =(residuals*residuals).sum()
    errors[l1_penalty] = rss

In [19]:
errors

{10.0: 398213327300134.44,
 31.622776601683793: 399041900253348.06,
 100.0: 429791604072558.25,
 316.22776601683796: 463739831045119.56,
 1000.0: 645898733633810.1,
 3162.2776601683795: 1222506859427156.8,
 10000.0: 1222506859427156.8,
 31622.776601683792: 1222506859427156.8,
 100000.0: 1222506859427156.8,
 316227.76601683791: 1222506859427156.8,
 1000000.0: 1222506859427156.8,
 3162277.6601683795: 1222506859427156.8,
 10000000.0: 1222506859427156.8}

In [20]:
#Report which L1 penalty produced the lower RSS on VALIDATION.
min(errors.items(),key=lambda x:x[1])

(10.0, 398213327300134.44)

#6. Quiz Question: Which was the best value for the l1_penalty, i.e. which value of l1_penalty produced the lowest RSS on VALIDATION data?


In [21]:
min(errors.items(),key=lambda x:x[1])

(10.0, 398213327300134.44)

In [22]:
#7. Now that you have selected an L1 penalty, compute the RSS on TEST data for the model with the best L1 penalty.
model_test = linear_model.Lasso(alpha=10.0,normalize=True)
model_test.fit(training[all_features],training['price'])
predictions_test =model_test.predict(testing[all_features])
residuals_test=predictions_test-testing['price']
rss =(residuals**2).sum()
print rss

1.22250685943e+15


#8. Quiz Question: Using the best L1 penalty, how many nonzero weights do you have? Count the number of nonzero coefficients first, and add 1 if the intercept is also nonzero. A succinct way to do this is



In [23]:
np.count_nonzero(model_test.coef_) + np.count_nonzero(model_test.intercept_)

15

#9. What if we absolutely wanted to limit ourselves to, say, 7 features? 
This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

You are going to implement a simple, two phase procedure to achieve this goal:
	
    •Explore a large range of ‘l1_penalty’ values to find a narrow region of ‘l1_penalty’ values where models are likely to have the desired number of non-zero weights.
	
    •Further explore the narrow region you found to find a good value for ‘l1_penalty’ that achieves the desired sparsity. Here, we will again use a validation set to choose the best value for ‘l1_penalty’.


In [24]:
#10.  Assign 7 to the variable ‘max_nonzeros’.
max_nonzeros=7

#11. Exploring large range of l1_penalty
For l1_penalty in np.logspace(1, 4, num=20):
	•	Fit a regression model with a given l1_penalty on TRAIN data. Add "alpha=l1_penalty" and "normalize=True" to the parameter list.


In [25]:
l1_penalty_values = np.logspace(1, 4, num=20)
model_weights={}
non_zero_weights={}
non_zero_list=[]

for l1_penalty in l1_penalty_values:
    model_new = linear_model.Lasso(alpha=l1_penalty,normalize=True)
    model_new.fit(training[all_features],training['price'])
    model_weights[l1_penalty]= model_new.coef_
    non_zero_weights[l1_penalty] = np.count_nonzero(model_new.coef_) + np.count_nonzero(model_new.intercept_) 
    

In [26]:
#Extract the weights of the model and count the number of nonzeros. Take account of the intercept as we did in 
# #8, adding 1 whenever the intercept is nonzero. Save the number of nonzeros to a list.
model_weights.keys()[0],model_weights.values()[0]

(42.813323987193932,
 array([ -1.81523764e+04,   0.00000000e+00,   3.82616260e+04,
          3.17143955e+02,  -1.43834130e+04,   0.00000000e+00,
         -2.01250123e+02,   0.00000000e+00,   5.42514210e+03,
          5.88437040e+05,   4.17071067e+04,   1.35251932e+04,
          1.23306929e+05,   0.00000000e+00,   0.00000000e+00,
         -3.35471851e+03,   1.37121819e+00]))

In [27]:
non_zero_weights

{10.0: 15,
 14.384498882876629: 15,
 20.691380811147901: 15,
 29.763514416313178: 15,
 42.813323987193932: 13,
 61.584821106602639: 12,
 88.586679041008225: 11,
 127.42749857031335: 10,
 183.29807108324357: 7,
 263.66508987303581: 6,
 379.26901907322497: 6,
 545.55947811685144: 6,
 784.75997035146065: 5,
 1128.8378916846884: 3,
 1623.776739188721: 3,
 2335.7214690901214: 2,
 3359.8182862837812: 1,
 4832.9302385717519: 1,
 6951.9279617756056: 1,
 10000.0: 1}

#12. Out of this large range, we want to find the two ends of our desired narrow range of l1_penalty. At one end, we will have l1_penalty values that have too few non-zeros, and at the other end, we will have an l1_penalty that has too many non-zeros.
More formally, find:
	•	The largest l1_penalty that has more non-zeros than ‘max_nonzeros’ (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)Store this value in the variable ‘l1_penalty_min’ (we will use it later)
	•	The smallest l1_penalty that has fewer non-zeros than ‘max_nonzeros’ (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)Store this value in the variable ‘l1_penalty_max’ (we will use it later)
Hint: there are many ways to do this, e.g.:
	•	Programmatically within the loop above
	•	Creating a list with the number of non-zeros for each value of l1_penalty and inspecting it to find the appropriate boundaries.


In [1]:
#The largest l1_penalty that has more non-zeros than max_nonzeros (max_nonzeros = 7 as we have set it )
#(if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)


In [28]:
l1_penalty_min = max( [k for (k,v) in non_zero_weights.items() if v > max_nonzeros])
l1_penalty_max = min(  [k for (k,v) in non_zero_weights.items() if v < max_nonzeros]) 
print(l1_penalty_min, l1_penalty_max)

(127.42749857031335, 263.66508987303581)


In [115]:
#. Quiz Question: What values did you find for l1_penalty_min and l1_penalty_max?
l1_penalty_min,l1_penalty_max

(127.42749857031335, 263.66508987303581)

#14. Exploring narrower range of l1_penalty
We now explore the region of l1_penalty we found: between ‘l1_penalty_min’ and ‘l1_penalty_max’. 
We look for the L1 penalty in this range that produces exactly the right number of nonzeros and also minimizes RSS on the VALIDATION set.

For l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
	
    •	Fit a regression model with a given l1_penalty on TRAIN data. As before, use "alpha=l1_penalty" and "normalize=True".
	
    •	Measure the RSS of the learned model on the VALIDATION set
Find the model that the lowest RSS on the VALIDATION set and has sparsity equal to ‘max_nonzeros’. (Again, take account of the intercept when counting the number of nonzeros.)

In [29]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)
model_weights_rss={}
non_zero_weights_rss={}

for l1_penalty in l1_penalty_values:
    model_new = linear_model.Lasso(alpha=l1_penalty,normalize=True)
    model_new.fit(training[all_features],training['price'])
    predictions=model_new.predict(validation[all_features] )
    residuals = predictions - validation['price']
    rss=sum(residuals**2)
    model_weights_rss[l1_penalty]= rss
    non_zero_weights_rss[l1_penalty] = np.count_nonzero(model_new.coef_) + np.count_nonzero(model_new.intercept_) 
    

In [30]:
model_weights_rss

{127.42749857031335: 435374677102678.38,
 134.59789811256189: 437009229124473.44,
 141.76829765481045: 438236128386911.62,
 148.93869719705901: 439158937799659.75,
 156.10909673930755: 440037365263316.94,
 163.27949628155611: 440777489641605.25,
 170.44989582380464: 441566698090139.19,
 177.6202953660532: 442406413188665.06,
 184.79069490830176: 443296716874313.19,
 191.96109445055032: 444239780526140.94,
 199.13149399279888: 445230739842614.19,
 206.30189353504741: 446268896864705.44,
 213.47229307729594: 447112919434642.31,
 220.6426926195445: 447998187851566.62,
 227.81309216179307: 448924706673256.0,
 234.98349170404163: 449892475899711.0,
 242.15389124629019: 450901498778122.88,
 249.32429078853872: 451952426654984.69,
 256.49469033078725: 453043924367600.69,
 263.66508987303581: 454176669662634.56}

In [31]:
non_zero_weights_rss

{127.42749857031335: 10,
 134.59789811256189: 10,
 141.76829765481045: 8,
 148.93869719705901: 8,
 156.10909673930755: 7,
 163.27949628155611: 7,
 170.44989582380464: 7,
 177.6202953660532: 7,
 184.79069490830176: 7,
 191.96109445055032: 7,
 199.13149399279888: 7,
 206.30189353504741: 6,
 213.47229307729594: 6,
 220.6426926195445: 6,
 227.81309216179307: 6,
 234.98349170404163: 6,
 242.15389124629019: 6,
 249.32429078853872: 6,
 256.49469033078725: 6,
 263.66508987303581: 6}

#15. Quiz Question: What value of l1_penalty in our narrow range has the lowest RSS on the VALIDATION set and has sparsity equal to ‘max_nonzeros’?

In [34]:
# and has sparsity equal to ‘max_nonzeros’ i.e max_nonzeros=7
l1_penalty_min = min([k for (k,v) in non_zero_weights_rss.items() if v==max_nonzeros])
l1_penalty_min

156.10909673930755

In [113]:
#16. Quiz Question: What features in this model have non-zero coefficients?

In [35]:
model_new = linear_model.Lasso(alpha=156.10909673930755,normalize=True)
model_new.fit(training[all_features],training['price'])


Lasso(alpha=156.109096739, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [36]:
np.count_nonzero(model_new.coef_) + np.count_nonzero(model_new.intercept_)

7

In [37]:
features_coef_greater_than_zero=[]
for i in range(len(all_features)):
    if(model_new.coef_[i]>0):
        features_coef_greater_than_zero.append(all_features[i])

print features_coef_greater_than_zero
print ''


['bathrooms', 'sqft_living', 'waterfront', 'view', 'grade']

